In [1]:
import os
import glob
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
# from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D, SeparableConv2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.preprocessing import image


In [2]:
# Custom CNN with Attention
def custom_cnn(input_shape, num_classes):
    inputs = Input(shape=input_shape+[3])

    # Block 1: Convolutional block with Attention
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    # x = BatchNormalization()(x)
    # x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    # x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # x = Dropout(0.3)(x)

    # Block 2: Advanced Convolutional Block (Separable Conv)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    # x = SeparableConv2D(64, (3, 3), padding='same', activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = SeparableConv2D(64, (3, 3), padding='same', activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    # x = Dropout(0.4)(x)

    # Block 3: Deeper Conv Block with Global Pooling
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    # x = BatchNormalization()(x)

    # Global Average Pooling and Fully Connected Layers
    x = GlobalAveragePooling2D()(x)

    x = Flatten()(x)
    # x = Dense(256, activation='relu')(x)
    # x = Dropout(0.5)(x)

    x = Dense(128, activation='relu')(x)
    # x = Dropout(0.5)(x)

    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.summary()

    return model

In [3]:
train_data_path = os.path.join('..', 'Dataset', 'CUB_data', 'data', 'train') 
valid_data_path = os.path.join('..', 'Dataset', 'CUB_data', 'data', 'test') 

# train_data_agumentation = ImageDataGenerator(rescale = 1./255,
#                                              zoom_range = 0.1,
#                                              rotation_range = 10,
#                                              width_shift_range = 0.1,
#                                              height_shift_range = 0.1
#                                              )

train_data_agumentation = ImageDataGenerator(rescale=1./255,  # Normalize pixel values
                                            shear_range=0.2,
                                            zoom_range=0.2,
                                            horizontal_flip=True,
                                            rotation_range=20,
                                            width_shift_range=0.2,
                                            height_shift_range=0.2,
                                            validation_split=0.2  # 20% for validation
                                        )

val_data_agumentation = ImageDataGenerator(rescale = 1./255)

# load training data
train_data = train_data_agumentation.flow_from_directory(directory = train_data_path,
                                                         target_size = (224,224),
                                                         class_mode = 'categorical',
                                                         batch_size = 8)

val_data = val_data_agumentation.flow_from_directory(directory = valid_data_path,
                                                     target_size = (224,224),
                                                     class_mode = 'categorical',
                                                     batch_size = 8)

Found 4156 images belonging to 2 classes.
Found 893 images belonging to 2 classes.


In [4]:
checkpoint_dir = 'checkpoints_CUB_CNN_models'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_filepath = os.path.join(checkpoint_dir,
                                   "model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}_val_loss_{val_loss:.2f}.keras")

checkpoint = ModelCheckpoint(filepath = checkpoint_filepath,
                             monitor = 'val_loss',
                             verbose = 0,
                             save_best_only = True,
                             save_weights_only = False,
                             mode = 'auto')

early = EarlyStopping(monitor = 'val_loss',
                      min_delta = 0,
                      patience = 10,
                      verbose = 0,
                      mode = 'auto')

reduceLR = ReduceLROnPlateau(monitor = "val_loss",
                             factor = 0.1,
                             patience = 5,
                             verbose = 0,
                             mode = "auto",
                             min_delta = 0.0001,
                             cooldown = 0,
                             min_lr = 0.0)

csv_logger = CSVLogger(os.path.join(checkpoint_dir, 'training.log'))

callbacks_list = [checkpoint, early, reduceLR, csv_logger] #, early 


In [5]:
image_size = [224, 224]
num_classes = len(glob.glob(train_data_path+'/*'))

model = custom_cnn(image_size, num_classes)

# Step 5: Compile the model
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

# Step 6: Train the model
history = model.fit(
                    train_data,  # Training data
                    epochs = 100,  # Number of epochs
                    batch_size = 8,
                    validation_data = val_data,  # Validation data
                    callbacks = callbacks_list
                )

# Save the trained model
save_model_path = os.path.join(checkpoint_dir, 'CUB_cnn_model.h5')
model.save(save_model_path)
print(f"Model saved as {save_model_path}.")




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 128)       7385

KeyboardInterrupt: 

In [ ]:
# accuracies
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.savefig('CUB_CNN_model_accuracy_and_val_accuracy.png', dpi=200)
plt.show()

# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('CUB_CNN_model_loss_and_val_loss.png', dpi=200)
plt.show()